In [1]:
import math
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

import GPy

import time as t

In [2]:
import pandas as pd
import sys
from env_pybullet_gen3 import env_pybullet_kin_gen3



In [3]:
#Create a experiment env
env = env_pybullet_kin_gen3(no_zeros = True,Excel_path_Okay_tcp = "./positions_from_joints_19.xlsx",time_step=0.05,home_angles = [-0.207226464676801,1.5689993219813,-1.01515387451347,-2.45271819663908,2.00795352004673,1.91098991659003,-0.831045149646278])
env.robot.visual_inspection = False

#Initially parameters of the urdf

#Make maxvels closer to the reality
#both have to be modified
env.max_vel = [168,151,128,94,210,48,189]
env.original_parameters_df ["max_vel"]=env.max_vel
print(env.original_parameters_df["max_vel"])
print(env.original_parameters_df)


print('observation space:', env.observation_space) #states, There is only 1 state constant
env.update_parameters_to_modify(["mass","max_vel","kp","ki","kd","force_x_one","Ixx","Iyy","Izz"])
print('action space:', env.action_space) #parameters, number of parameters choose to tune, continuous
print('original action:', env.action_original()) #parameters, number of parameters choose to tune, continuous




hola
../Simulation_Pybullet/models/urdf/JACO3_URDF_V11.urdf
Robot launched
hola
(7, 12)
(7, 17)
0    168
1    151
2    128
3     94
4    210
5     48
6    189
Name: max_vel, dtype: int64
       mass damping       Ixx       Iyy       Izz   kp   ki   kd  max_vel  \
0  1.377353       0  0.004801  0.004755  0.002283  0.1  0.0  0.0      168   
1  1.163667       0  0.008419  0.001920  0.008361  0.1  0.0  0.0      151   
2  1.163660       0  0.007545  0.007487  0.001921  0.1  0.0  0.0      128   
3  0.930287       0  0.006410  0.001380  0.006518  0.1  0.0  0.0       94   
4  0.678106       0  0.001680  0.001506  0.000826  0.1  0.0  0.0      210   
5  0.678106       0  0.001938  0.000827  0.001763  0.1  0.0  0.0       48   
6  0.500657       0  0.000775  0.000585  0.000975  0.1  0.0  0.0      189   

   force_x_one  
0            1  
1            1  
2            1  
3            1  
4            1  
5            1  
6            1  
observation space: 1
mass okey
max_vel okey
kp okey
ki okey


In [4]:
#Create a first search
pop_size = 70*20
sigma = 0.3
original_action = np.array(env.action_original())
np.random.seed(0)
Create = False
rewards = []
actions = []

if(Create ==True):
    #Generate new population weights to test
    weights_pop = [(sigma*np.random.randn(env.action_space)) for i in range(pop_size)]
    for weights in weights_pop:
                    action=np.add(np.multiply(weights,original_action),original_action)
                    actions.append(action)
                    rewards.append(env.step_tcp_rishabh(action))
    actions = np.array(actions)
    rewards = np.array(rewards).reshape(actions.shape[0],1)
    
    i_data = np.hstack((actions,rewards))
    np.save("i_data.npy",i_data)
else:
    i_data = np.load("i_data.npy")
    actions = i_data[:,:-1]
    rewards = i_data[:,-1:]


In [5]:
print("actions")
print(actions.shape)
print(actions)
print("rewards")
print(rewards.shape)
print(rewards)

actions
(1400, 63)
[[2.10626984e+00 1.30336192e+00 1.50533547e+00 ... 6.59419705e-04
  1.57288567e-03 7.37282918e-04]
 [6.64042844e-01 1.22560648e+00 1.02339908e+00 ... 1.15477545e-03
  1.39569038e-03 9.31393856e-04]
 [1.19754499e+00 1.80924515e+00 1.39835676e+00 ... 5.52380219e-04
  1.79065083e-03 7.58809772e-04]
 ...
 [1.97407477e+00 6.37037773e-01 1.16377190e+00 ... 1.12702430e-03
  1.26605998e-03 1.19178768e-03]
 [7.96462938e-01 1.29488958e+00 9.74594365e-01 ... 9.36204322e-04
  1.23476868e-03 1.05441314e-03]
 [1.28513665e+00 6.25185039e-01 1.18711875e+00 ... 7.00949285e-04
  2.19622534e-03 1.02368897e-03]]
rewards
(1400, 1)
[[-0.15615385]
 [-0.17645011]
 [-0.32277424]
 ...
 [-0.21721271]
 [-0.19800912]
 [-0.16752433]]


In [6]:
#Create a Gaussian model, which models the relation between parameters and score
kernel = GPy.kern.Linear(actions.shape[1], ARD=1)
#kernel = GPy.kern.RBF_inv(X.shape[1], ARD=1)

model = GPy.models.GPRegression(actions, rewards, kernel)

model.optimize(optimizer='scg', max_iters=10**2)

In [7]:
predictions =  model.predict(actions)

error = abs(np.array(predictions[0])-rewards)
error_max = max(error)
error_mean = error.mean(axis=0)

print("error")
print(error)
print("error_max")
print(error_max)
print("error_mean")
print(error_mean)

error
[[0.02282811]
 [0.03597479]
 [0.02785162]
 ...
 [0.0731605 ]
 [0.02331862]
 [0.02780115]]
error_max
[0.26048575]
error_mean
[0.02714948]


In [11]:
# Get the best action from regressed model

def Get_best_action_from_model(env,model,sigma = 0.3, population = 70*10**3,n_elite = 1):
    if(population>70*10**3):
        print("More data fill the memory, and creates and error, used only 70*10**3")
        population = 70*10**3
    
    weights_pop_model = [(sigma*np.random.randn(env.action_space)) for i in range(population)]
    
    actions_model = []
    for weights in weights_pop_model:
        action=np.add(np.multiply(weights,original_action),original_action)
        actions_model.append(action)
    actions_model = np.array(actions_model)
    
    prediction = model.predict(actions_model)
    rewards_model = np.array(prediction[0])
    
    elite_idxs = rewards_model.argsort()[-n_elite:]
    elite_actions = [actions_model[i] for i in elite_idxs]

    #Set the best weight as the mean of the best ones 

    best_action = np.array(elite_actions).mean(axis=0)
    best_rewards = rewards_model.argsort()[-n_elite:]
    
    return best_action,best_rewards
    
    

In [12]:
best_action,best_rewards = Get_best_action_from_model(env,model)
print(best_rewards)
print(best_action)


[[0]]
[[1.74888546e+00 8.71340795e-01 1.30801155e+00 7.94931450e-01
  7.78222445e-01 6.91341992e-01 4.69188843e-01 1.02012704e+02
  2.37780727e+02 9.19188266e+01 1.13417027e+02 1.73919629e+02
  7.00817512e+01 1.72033563e+02 1.13274482e-01 8.17524916e-02
  8.45878640e-02 1.06371286e-01 7.44470800e-02 7.29366067e-02
  1.32978328e-01 7.75892079e-02 1.96693120e-02 4.85411772e-02
  6.21368640e-02 5.96542087e-02 5.19383184e-02 4.78997213e-02
  5.69843384e-02 3.44262844e-02 5.51881573e-02 3.39141982e-02
  5.26951122e-02 4.30974703e-02 5.80267862e-02 7.99847557e-01
  6.97672933e-01 5.11679629e-01 1.44085338e+00 8.91453615e-01
  1.12878321e+00 1.11557190e+00 4.59736268e-03 8.14862003e-03
  6.79916974e-03 4.44293193e-03 1.59292106e-03 2.26721121e-03
  6.52364597e-04 5.33631470e-03 1.97016972e-03 1.04777438e-02
  1.70828555e-03 1.85651798e-03 1.03863995e-03 8.19729445e-04
  2.08463231e-03 6.23379061e-03 1.69591499e-03 8.07849726e-03
  1.13669271e-03 1.75002324e-03 1.29463944e-03]]


In [6]:
#Cross Entrophy Method, to choose the weights

# In my case where only 1 action,and that it's apply the parameters do another step doesn't change anything due to the state doesn't vary
# For this reason max_t and gama doesn't make sense, so I set them to max_t = 1 and gamma to 0
def cem_no_net(n_iterations=700, max_t=1, gamma=0.0, print_every=50, pop_size=env.action_space, elite_frac=10/(env.action_space*2), sigma=0.3,sigma_reduction_every_print = 0.65, per_one = True ):
    """PyTorch implementation of the cross-entropy method.
        
    Params
    ======
        n_iterations (int): maximum number of training iterations
        max_t (int): maximum number of timesteps per episode
        gamma (float): discount rate
        print_every (int): how often to print average score (over last 100 episodes)
        pop_size (int): size of population at each iteration
        elite_frac (float): percentage of top performers to use in update
        sigma (float): standard deviation of additive noise
        per_one (boolean): to determine if the output is in per one or not
    """
    #Numbers of elements that you keep as the better ones
    n_elite=int(pop_size*elite_frac)
    
    #scores doble end queee , from iterations size * 0.1
    scores_deque = deque(maxlen=int(n_iterations*0.1))
    #intial scores empty
    scores = []
    #Save actions to see how they evolve
    best_actions = []
    #Select a seed to make the results the same every test, not depending on the seed
    np.random.seed(0)
    #Initial best weights, are from 0 to 1, it's good to be small the weights, but they should be different from 0.
    # small to avoid overfiting , different from 0 to update them
    
    if (per_one == True):
        best_weight = sigma*np.random.randn(env.action_space)
        original_action = np.array(env.action_original())
    else:
        best_weight = np.add(sigma*np.random.randn(env.action_space),env.action_original())

    #Each iteration, modify  + (from 0 to 1) the best weight randomly
    #Computes the reward with these weights
    #Sort the reward to get the best ones
    # Save the best weights
    # the Best weight it's the mean of the best one
    #compute the main reward of the main best rewards ones
    #this it's show to evalute how good its
    
    for i_iteration in range(1, n_iterations+1):
        
        #Generate new population weights, as a mutation of the best weight to test them
        weights_pop = [best_weight + (sigma*np.random.randn(env.action_space)) for i in range(pop_size)]
        
        #Compute the parameters and obtain the rewards for each of them
        #print("iteration "+str(i_iteration))
        if (per_one == True):
            rewards=[]
            for weights in weights_pop:
                #print("New weights")
                #print(weights)
                #t.sleep(1000)
                action=np.add(np.multiply(weights,original_action),original_action)
                rewards.append( env.step_tcp_rishabh(action) )
            rewards = np.array(rewards)
            
        else:
            rewards=[]
            for weights in weights_pop:
                rewards.append(env.step_tcp_rishabh(weights) )
            rewards = np.array(rewards)
        print("rewards" + str(i_iteration))
        print(rewards)
        #print("\n")
        
        #Sort the rewards to obtain the best ones
        elite_idxs = rewards.argsort()[-n_elite:]
        elite_weights = [weights_pop[i] for i in elite_idxs]
        
        #Set the best weight as the mean of the best ones 
       
        best_weight = np.array(elite_weights).mean(axis=0)
        
        #Get the reward with this new weight
        
        if (per_one == True):
            action = np.add(np.multiply(best_weight,original_action),original_action)
            #print("action vel")
            #print(action[8])
            #print("action kp")
            #print(action[15])
            #time.sleep(1000)
            best_actions.append(action)
            reward = env.step_tcp_rishabh(action)
            print("reward")
            print(reward)
            print("\n")
        else:
            best_actions.append(best_weight)
            reward = env.step_tcp_rishabh(best_weight)
        scores_deque.append(reward)
        scores.append(reward)
        
        #save the check point
        env.save_parameters("./Parameters_train_tcp_euc_rishabh.xlsx")
        
        if i_iteration % print_every == 0:
            sigma = sigma * sigma_reduction_every_print
            print('Episode {}\tAverage Score: {:.2f}'.format(i_iteration, np.mean(scores_deque)))

        if np.mean(scores_deque)>=0.0:
            print('\nEnvironment solved in {:d} iterations!\tAverage Score: {:.2f}'.format(i_iteration-n_iterations*0.1, np.mean(scores_deque)))
            break
    return scores,best_actions


In [7]:
#Execute the cross entrophy method with default Values
#scores = cem()


#To don't ask the GPU as much reduce the pop_size, it's the amount of elemts try
scores,best_actions = cem_no_net()
# 
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()
    

rewards1
[-0.16397515 -0.28245656 -0.17721985 -0.16621907 -0.15203679 -0.27111682
 -0.22933904 -0.17850869 -0.17382578 -0.15260351 -0.25045082 -0.15665328
 -0.15505304 -0.18666162 -0.18071532 -0.23407996 -0.1801245  -0.15165878
 -0.24291922 -0.1623494  -0.16798387 -0.12790071 -0.18694454 -0.15637045
 -0.14879577 -0.15232597 -0.18251678 -0.24859745 -0.20067303 -0.26525601
 -0.20688806 -0.19095564 -0.18090049 -0.17862065 -0.1748199  -0.15024075
 -0.18638866 -0.20278096 -0.17812159 -0.15466958 -0.16383426 -0.14786135
 -0.23410482 -0.20201706 -0.20068485 -0.22603929 -0.17516261 -0.17693484
 -0.1747468  -0.16192645 -0.16512089 -0.17199096 -0.19613955 -0.17332991
 -0.14372354 -0.28094054 -0.16108797 -0.1536825  -0.43536958 -0.13662217
 -0.18750237 -0.1543218  -0.15458316]
reward
-0.15340520134665103


rewards2
[-0.1448248  -0.25428389 -0.18949537 -0.14887906 -0.1477281  -0.16535738
 -0.17268272 -0.14269907 -0.16584637 -0.16365152 -0.16066453 -0.14294508
 -0.14155905 -0.20197002 -0.16159327 -

KeyboardInterrupt: 

In [ ]:

# plot the last scores zoom
fig = plt.figure()
zoom= 300
ax = fig.add_subplot(111)
plt.plot(np.arange(1, zoom+1), scores[-zoom:])
plt.ylabel('Score zoom')
plt.xlabel('Episode #')
plt.show()

In [ ]:
# plot actions
best_actions_np = np.array(best_actions)
joint = 1
for i in range(len(env.parameters_to_modify)) :
    parameter = env.parameters_to_modify[i]
    figures = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(1, best_actions_np.shape[0]+1), best_actions_np[:,joint+i*7])
    plt.ylabel(parameter+" Joint"+str(joint))
    plt.xlabel('Episode #')
    plt.show()

In [ ]:
# load the weights from file
# Not working know


#state = env.reset()
env = env_pybullet_kin_gen3(no_zeros = True,Excel_path_Okay_tcp = "./positions_from_joints_19.xlsx",time_step=0.05,home_angles = [-0.207226464676801,1.5689993219813,-1.01515387451347,-2.45271819663908,2.00795352004673,1.91098991659003,-0.831045149646278])
env.robot.visual_inspection = False

#Make maxvels closer to the reality
env.max_vel = [168,151,128,94,210,48,189]
env.original_parameters_df ["max_vel"]=env.max_vel
env.modified_parameters_df ["max_vel"]=[168,151,128,94,210,48,189]

env.update_parameters_to_modify(["mass","max_vel","kp","ki","kd","force_x_one","Ixx","Iyy","Izz","damping"])
env.robot.visual_inspection = False
env.modified_parameters_df = env.create_df_from_Excel("./Parameters_train_tcp_euc_rishabh.xlsx")


t.sleep(0.02)
action = env.action_modified()
action = np.array(action)
print('original action:', env.action_original()) #parameters, number of parameters choose to tune, continuous
print("trained",action)
reward = env.step_tcp_rishabh(action)
print("reward")
print(reward)



In [ ]:
#Convert to excel
a = env.df_avg.to_numpy()
print(a[:,5])
env.df_avg.to_excel("./Train_parameters_result_tcp_euc_rishabh.xlsx")

In [ ]:
env.original_parameters_df

In [ ]:
# load the weights from file
# Not working know


#state = env.reset()
env = env_pybullet_kin_gen3(no_zeros = True,Excel_path_Okay_tcp = "./positions_from_joints_19.xlsx",time_step=0.05,home_angles = [-0.207226464676801,1.5689993219813,-1.01515387451347,-2.45271819663908,2.00795352004673,1.91098991659003,-0.831045149646278])
env.robot.visual_inspection = False

#Make maxvels closer to the reality
env.max_vel = [168,151,128,94,210,48,189]
env.original_parameters_df ["max_vel"]=env.max_vel
env.modified_parameters_df ["max_vel"]=[168,151,128,94,210,48,189]

env.update_parameters_to_modify(["mass","max_vel","kp","ki","kd","force_x_one","Ixx","Iyy","Izz"])
env.robot.visual_inspection = False
env.modified_parameters_df = env.original_parameters_df


t.sleep(0.02)
action = env.action_modified()
action = np.array(action)
print(action)
reward = env.step_tcp_rishabh(action)

print("reward")
print(reward)

In [ ]:
#Convert to excel
a = env.df_avg.to_numpy()
print(a[:,5])
env.df_avg.to_excel("./Original_parameters_result_tcp_euc_rishabh.xlsx")